In [2]:
# Установи: pip install groundingdino-py
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from groundingdino.util.inference import load_model, load_image, predict
from sam2.sam2_image_predictor import SAM2ImagePredictor

def simple_grounding_sam2(image_path, text_prompt):
    """Простой GroundingDINO + SAM2"""
    
    print(f"🔍 Ищем: {text_prompt}")
    
    # 1. Загружаем модели
    print("📥 Загружаем GroundingDINO...")
    grounding_model = load_model(
        "groundingdino/config/GroundingDINO_SwinT_OGC.py", 
        "weights/groundingdino_swint_ogc.pth"
    )
    
    print("📥 Загружаем SAM2...")
    sam_model = SAM2ImagePredictor.from_pretrained("facebook/sam2-hiera-tiny")
    
    # 2. Загружаем изображение
    image_source, image = load_image(image_path)
    
    # 3. Ищем объекты по тексту
    print("🎯 Ищем объекты...")
    boxes, scores, phrases = predict(
        model=grounding_model,
        image=image,
        caption=text_prompt,
        box_threshold=0.3,
        text_threshold=0.25
    )
    
    print(f"✅ Найдено: {len(boxes)} объектов")
    for i, (phrase, score) in enumerate(zip(phrases, scores)):
        print(f"   {i+1}. {phrase} (уверенность: {score:.3f})")
    
    # 4. Сегментируем найденные объекты
    print("✂️ Сегментируем...")
    sam_model.set_image(image_source)
    
    all_masks = []
    
    for i, box in enumerate(boxes):
        # Конвертируем координаты
        H, W = image_source.shape[1], image_source.shape[2]
        x1, y1, x2, y2 = box * torch.tensor([W, H, W, H])
        bbox = np.array([x1, y1, x2, y2])
        
        # Сегментируем объект
        masks, mask_scores, _ = sam_model.predict(box=bbox[None, :])
        all_masks.append(masks[0])
    
    # 5. Показываем результат
    print("🖼️ Показываем результат...")
    
    # Конвертируем изображение для отображения
    image_display = image_source[0].permute(1, 2, 0).cpu().numpy()
    
    plt.figure(figsize=(12, 8))
    plt.imshow(image_display)
    
    # Рисуем маски
    for i, mask in enumerate(all_masks):
        # Случайный цвет для каждой маски
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
        h, w = mask.shape[-2:]
        mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
        plt.imshow(mask_image)
        
        # Подпись объекта
        center_y, center_x = np.where(mask)
        if len(center_x) > 0 and len(center_y) > 0:
            text_x = np.mean(center_x)
            text_y = np.mean(center_y)
            
            plt.text(text_x, text_y, f"{phrases[i]}\n({scores[i]:.2f})", 
                    fontsize=12, fontweight='bold', color='white',
                    bbox=dict(boxstyle="round,pad=0.3", facecolor='black', alpha=0.8),
                    ha='center', va='center')
    
    plt.axis('off')
    plt.title(f"Найдено: {text_prompt}\n{len(boxes)} объектов", fontsize=14)
    plt.tight_layout()
    plt.show()
    
    print("🎉 Готово!")

# 🎯 ПРОСТЫЕ ПРИМЕРЫ:

print("=== ПРИМЕР 1: Люди и техника ===")
simple_grounding_sam2(
    "/home/oleg/projects/ML/CLIP/images/0d354ad89e92986b19b10a8ac2797dfb.jpg",
    "person . laptop . cup"
)

print("\n=== ПРИМЕР 2: Только человек ===")
simple_grounding_sam2(
    "/home/oleg/projects/ML/CLIP/images/0d354ad89e92986b19b10a8ac2797dfb.jpg",
    "person"
)

print("\n=== ПРИМЕР 3: Растения и посуда ===")
simple_grounding_sam2(
    "/home/oleg/projects/ML/CLIP/images/0d354ad89e92986b19b10a8ac2797dfb.jpg",
    "plant . cup"
)

ModuleNotFoundError: No module named 'groundingdino'

In [1]:
# pip install groundingdino-py

import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from groundingdino.util.inference import load_model, load_image, predict
from sam2.sam2_image_predictor import SAM2ImagePredictor

def grounding_sam2_demo(image_path, text_prompt):
    """GroundingDINO + SAM2 по текстовому промпту"""
    
    print(f"🎯 Ищем: '{text_prompt}'")
    
    # 1. Загрузка GroundingDINO
    print("1. Загружаем GroundingDINO...")
    grounding_model = load_model(
        "groundingdino/config/GroundingDINO_SwinT_OGC.py", 
        "weights/groundingdino_swint_ogc.pth"  # Автоскачается
    )
    
    # 2. Загрузка изображения
    image_source, image = load_image(image_path)
    
    # 3. Детекция по тексту
    print("2. Детекция по тексту...")
    boxes, logits, phrases = predict(
        model=grounding_model,
        image=image,
        caption=text_prompt,
        box_threshold=0.3,    # Порог уверенности bbox
        text_threshold=0.25   # Порог соответствия тексту
    )
    
    print(f"✅ Найдено объектов: {len(boxes)}")
    for i, (box, phrase, logit) in enumerate(zip(boxes, phrases, logits)):
        print(f"   {i+1}. {phrase} (уверенность: {logit:.3f})")
    
    # 4. Загрузка SAM2
    print("3. Загружаем SAM2...")
    sam_predictor = SAM2ImagePredictor.from_pretrained("facebook/sam2-hiera-tiny")
    
    # 5. Сегментация найденных объектов
    print("4. Сегментация...")
    sam_predictor.set_image(image_source)
    
    all_masks = []
    object_info = []
    
    for i, (box, phrase, logit) in enumerate(zip(boxes, phrases, logits)):
        # Конвертируем bbox в формат SAM2
        H, W = image_source.shape[1], image_source.shape[2]
        x1, y1, x2, y2 = box * torch.tensor([W, H, W, H])
        bbox = np.array([x1, y1, x2, y2])
        
        masks, scores, _ = sam_predictor.predict(box=bbox[None, :])
        
        all_masks.append(masks[0])
        object_info.append({
            'mask': masks[0],
            'phrase': phrase,
            'confidence': logit,
            'score': scores[0]
        })
    
    # 6. Визуализация
    visualize_grounding_sam2(image_source, object_info, text_prompt)

def visualize_grounding_sam2(image, object_info, text_prompt):
    """Визуализация результатов"""
    
    plt.figure(figsize=(14, 8))
    plt.imshow(image[0].permute(1, 2, 0).cpu().numpy())
    
    for i, obj in enumerate(object_info):
        show_mask(obj['mask'], plt.gca(), random_color=True)
        
        mask_center = find_mask_center(obj['mask'])
        label = f"{obj['phrase']} ({obj['confidence']:.2f})"
        
        plt.text(mask_center[0], mask_center[1], label, 
                fontsize=10, fontweight='bold',
                bbox=dict(boxstyle="round,pad=0.3", 
                         facecolor='white', 
                         edgecolor='black', 
                         alpha=0.8),
                ha='center', va='center')
    
    plt.axis('off')
    plt.title(f"GroundingDINO+SAM2: '{text_prompt}'\n{len(object_info)} объектов", fontsize=14)
    plt.tight_layout()
    plt.show()

def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

def find_mask_center(mask):
    y_coords, x_coords = np.where(mask)
    if len(x_coords) > 0 and len(y_coords) > 0:
        center_x = np.mean(x_coords)
        center_y = np.mean(y_coords)
        return (center_x, center_y)
    else:
        return (0, 0)

# 🎯 Примеры использования:
print("=== Пример 1: Поиск конкретных объектов ===")
grounding_sam2_demo(
    "/home/oleg/projects/ML/CLIP/images/0d354ad89e92986b19b10a8ac2797dfb.jpg",
    "cup . laptop"  # 👈 Найди ТОЛЬКО чашку и ноутбук
)

print("\n=== Пример 2: Поиск по описанию ===")
grounding_sam2_demo(
    "/home/oleg/projects/ML/CLIP/images/0d354ad89e92986b19b10a8ac2797dfb.jpg", 
    "person . plant"  # 👈 Найди ТОЛЬКО человека и растение
)

ModuleNotFoundError: No module named 'groundingdino'